In [1]:
# GrovePi
import grovepi

# LCD
import decimal
import drivers.grove_rgb_lcd #import *
import drivers.grove_oled #import *

# Auxilary packages
import time, math, csv
import random as rd
import numpy as np

noiseTreshold = 700
distanceTreshold0 = 10
distanceTreshold1 = 600

# Connected devices
Devices = {
    'sound_sensor': 0,    
    'co2_sensor': 1,
    'photoresistor': 2,
    'led_red': 4,
    'led_green': 3,
    'led_yellow': -1,
    'dht_sensor': 7,
    'ultrasonic_ranger': 8
}

#sentences that will appear if a sensor theshold is exceeded
sentences_sound = [
        "I've heard a noise!", 
        "Who's there?", 
        'I can hear you!',
        'Who is making noise?'
        ]
sentences_distance0 = [
        'This is my eye.', 
        'Hey!', 
        'This is an ultrasonic sensor',
        'Proximity alert!'
        ]
sentences_distance1 = [
        'I can see you!', 
        'Who is there?', 
        'Someone is in front of me.',
        'Hello?'
        ]

In [2]:
if False:
    outfilename = 'readings.csv'
    with open(outfilename, 'w') as csvfile:
        print("writing to <%s>" % outfilename)
        fieldnames = ['starttime','distance','light','sound','hum','temp']

        starttime = time.time()
        sound = 55
        distance = 527    
        light = 600
        temp = 25.6
        hum = 20.2

        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerow({'starttime':starttime,'distance':distance,'light':light,'sound':sound,'hum':hum,'temp':temp})
    print("<%s> closed" % outfilename)

In [3]:
# OUTPUT devices
class LCD:                
    def __init__(self):
        self.dpR, self.dpG, self.dpB = 100, 100, 100
        self.dpText = ''
        self.busy = False
        self.pendingMessages = []
        
    class Message:
        def __init__(self, text, colour):
            self.text = text
            self.colour = colour
    
    def reset(self):
        self.deletePendingMessages()
        drivers.grove_rgb_lcd.setRGB(0,0,0)
        drivers.grove_rgb_lcd.setText('')        
        
    def updateColour(self):
        if not self.busy:
            drivers.grove_rgb_lcd.setRGB(self.dpR, self.dpG, self.dpB)     
        
    def updateColourVar(self,r,g,b):
        self.dpR, self.dpG, self.dpB = r, g, b
        self.updateColour()
        
    def defaultColour(self):
        self.dpR, self.dpG, self.dpB = 100,100,100  
        self.updateColour()
    
    def updateText(self,):
        if not self.busy:
            drivers.grove_rgb_lcd.setText(self.dpText)
        
    def updateTextVar(self, text=''):
        self.dpText = text
        self.updateText()
        
    def newMessage(self, text, colour=[10,10,10]):
        self.pendingMessages.append(self.Message(text,colour))
        
    def popMessage(self, message):
        drivers.grove_rgb_lcd.setText(message.text)
        drivers.grove_rgb_lcd.setRGB(message.colour[0], message.colour[1], message.colour[2])        
        try:
            self.pendingMessages.pop(0)
        except:
            print('Error: Message: List is already empty')
#         time.sleep(message.duration)
    
    def displayMessage(self):
        self.busy = True
        if len(self.pendingMessages) > 0:
            self.popMessage(self.pendingMessages[0])                
                
    def endDisplayMessage(self):
        self.busy = False
        self.updateColour()
        self.updateText()
        
    def deletePendingMessages(self):
        for i,x in enumerate(self.pendingMessages):
            self.pendingMessages.pop(i)

def resetLEDs():      
    grovepi.pinMode(Devices['led_red'],"OUTPUT")
    grovepi.pinMode(Devices['led_green'],"OUTPUT")
    grovepi.analogWrite(Devices['led_red'],0)
    grovepi.analogWrite(Devices['led_green'],0)    

In [4]:
lcd = LCD()

lcd.reset()
lcd.defaultColour()

unit_test = False

if unit_test:
    lcd.newMessage('hello0', colour=[10,200,200])
    lcd.newMessage('hello1', colour=[10,200,100])
    lcd.newMessage('hello2', colour=[200,100,10])

    print(lcd.pendingMessages)
    
    for i in lcd.pendingMessages:
        lcd.displayMessage()
        time.sleep(2)
        lcd.endDisplayMessage()

In [60]:
def mainEvent(fps = 10.0, lcd_fps = 0.1):    
    
    spf = 1 / fps
    lcdMain_spf = 1 / lcd_fps

    messageDuration = 3.0

    lcd = LCD()
#     resetLEDs()

    frametime = spf    
    lcdMain_frametime = lcdMain_spf
    lcdMain_starttime = time.time()    
    
    lcdMessage_starttime = 3.0
    lcdMessage_frametime = 3.0    
    
    messageInProgress = False
    
    lcd.defaultColour()
    
    while True:   
        try:
            if frametime >= spf:
                starttime = time.time()               
            
                try:
                    # reading from sensors and changing LEDS accordingly  
                    # ---sound                
                    sound = grovepi.analogRead(Devices['sound_sensor'])                
                    # ---light                
                    light = grovepi.analogRead(Devices['photoresistor'])  
                    # ---distance
                    distance = grovepi.ultrasonicRead(Devices['ultrasonic_ranger'])
                except TypeError:
                    sound,light,distance = -1,-1,-1
            
                # ---humidity & temperature
                [temp, hum] = grovepi.dht(Devices['dht_sensor'], 0)                
                # ---gas
#                 gas = grovepi.analogRead(Devices['gas_sensor'])  

                # LED reaction 
#                 grovepi.analogWrite(Devices['led_green'],int(light/4))
#                 grovepi.analogWrite(Devices['led_green'],int(sound/4))
                
#                 if distance < distanceTreshold0:
#                     grovepi.analogWrite(Devices['led_red'],255)
#                 else:   
#                     grovepi.analogWrite(Devices['led_red'],0)
                    
#                 if not math.isnan(light):                        
#                     grovepi.analogWrite(Devices['led_green'],255-int(light/4))
                    
                # write to CSV                
                t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(starttime))
                print('%s, %d, %d,%d, %1.f, %1.f' % (t,distance,light,sound,hum,temp), end='\r', flush=True)
                
                # message on LCD ?
                if not messageInProgress:
                    if sound > noiseTreshold:     
                        sentence = rd.randint(0,len(sentences_sound)-1)
                        lcd.newMessage(sentences_sound[sentence], colour=[10,200,200])

                    if distance < distanceTreshold0 and distance>0:
                        sentence = rd.randint(0,len(sentences_distance0)-1)
                        lcd.newMessage(sentences_distance0[sentence], colour=[200,200,10])
                    
                
                if len(lcd.pendingMessages) > 0:   

                    if not messageInProgress:
                        lcdMessage_starttime = time.time()
                        lcd.displayMessage() 
                        messageInProgress = True

                if lcdMessage_frametime >= messageDuration:
                    lcd.endDisplayMessage()     
                    messageInProgress = False
                    
                if lcdMain_frametime >= lcdMain_spf:
                    if not messageInProgress:
                        lcdMain_starttime = time.time() 
                        stringForLCD = 'lumi temp  humid%4d%5.1fC%5.1f%%' % (light, temp, hum)
#                         stringForLCD = 'lumi temp  humid'+str(int(light*999/1024))+'  '+str(temp)+'C '+str(hum)+'%'
                        lcd.updateTextVar(stringForLCD)
                        lcd.updateText()
                
            # FIXED TIME STEP technique
            frametime = time.time() - starttime
            lcdMain_frametime = time.time() - lcdMain_starttime
            lcdMessage_frametime = time.time() - lcdMessage_starttime
            # UPDATE LCD 
        except (KeyboardInterrupt, SystemExit):
#             resetLEDs()            
            lcd.reset()
            break
            
mainEvent(1)

In [59]:
unit_test = False
if unit_test:

    light = 653
    temp = 25.6
    hum = 20.2

#     stringForLCD = 'lumi temp  humid'+str(int(light*999/1024))+'  '+str(temp)+'C '+str(hum)+'%'
    stringForLCD = 'lumi temp  humid%4d%5.1fC%5.1f%%' % (light, temp, hum)
    lcd.updateTextVar(stringForLCD)
    lcd.updateText()

In [44]:
# for i in range(1023):
#     sound = grovepi.analogRead(Devices['sound_sensor'])
#     grovepi.analogWrite(Devices['led_green'],i)
#     if i%10==0:
#         time.sleep(0.3)
#     print('%d, %f' % (i,sound), end='\r', flush=True)

In [13]:
# # from grovepi import *
# # from grove_oled import *


# #Start and initialize the OLED
# oled_init()
# oled_clearDisplay()
# oled_setNormalDisplay()
# oled_setVerticalMode()
# time.sleep(.1)

# while True:
#     try:
#         [ temp,hum ] = dht(dht_sensor,1)		#Get the temperature and Humidity from the DHT sensor
#         string = "temp ="+ str(temp)+ "C\thumidity ="+ str(hum)+"%"
#         print(string, end='\r', flush=True)
#         t = str(temp)
#         h = str(hum)

#         oled_setTextXY(0,1)			#Print "WEATHER" at line 1
#         oled_putString("WEATHER")

#         oled_setTextXY(2,0)			#Print "TEMP" and the temperature in line 3
#         oled_putString("Temp:")
#         oled_putString(t+'C')

#         oled_setTextXY(3,0)			#Print "HUM :" and the humidity in line 4
#         oled_putString("Hum :")
#         oled_putString(h+"%")
#     except (IOError,TypeError) as e:
#         print("Error", end='\r', flush=True)



In [14]:
# import decimal
# from grovepi import *
# from drivers.grove_rgb_lcd import *


# dht_sensor_port = 7     # Connect the DHt sensor to port 7
# lastTemp = 0.1          # initialize a floating point temp variable
# lastHum = 0.1           # initialize a floating Point humidity variable
# tooLow = 62.0           # Lower limit in fahrenheit
# justRight = 68.0        # Perfect Temp in fahrenheit
# tooHigh = 74.0          # Temp Too high


# # Function Definitions
# def CtoF( tempc ):
#    "This converts celcius to fahrenheit"
#    tempf = round((tempc * 1.8) + 32, 2);
#    return tempf;

# def FtoC( tempf ):
#    "This converts fahrenheit to celcius"
#    tempc = round((tempf - 32) / 1.8, 2)
#    return tempc;

# def calcColorAdj(variance):     # Calc the adjustment value of the background color
#     "Because there is 6 degrees mapping to 255 values, 42.5 is the factor for 12 degree spread"
#     factor = 42.5;
#     adj = abs(int(factor * variance));
#     if adj > 255:
#         adj = 255;
#     return adj;

# def calcBG(ftemp):
#     "This calculates the color value for the background"
#     variance = ftemp - justRight;   # Calculate the variance
#     adj = calcColorAdj(variance);   # Scale it to 8 bit int
#     bgList = [0,0,0]               # initialize the color array
#     if(variance < 0):
#         bgR = 0;                    # too cold, no red
#         bgB = adj;                  # green and blue slide equally with adj
#         bgG = 255 - adj;
        
#     elif(variance == 0):             # perfect, all on green
#         bgR = 0;
#         bgB = 0;
#         bgG = 255;
        
#     elif(variance > 0):             #too hot - no blue
#         bgB = 0;
#         bgR = adj;                  # Red and Green slide equally with Adj
#         bgG = 255 - adj;
        
#     bgList = [bgR,bgG,bgB]          #build list of color values to return
#     return bgList;

# while True:

#     try:
#         temp = 0.01
#         hum = 0.01
#         [ temp,hum ] = dht(dht_sensor_port,0)       #Get the temperature and Humidity from the DHT sensor
#                                                     #Change the second parameter to 0 when using DHT (instead of DHT Pro)
#                                                     #You will get very large number values if you don't!
#         if (CtoF(temp) != lastTemp) and (hum != lastHum) and not math.isnan(temp) and not math.isnan(hum):
#                 print("lowC : ",FtoC(tooLow),"C\t\t","rightC  : ", FtoC(justRight),"C\t\t","highC : ",FtoC(tooHigh),"C") # comment these three lines
#                 print("lowF : ",tooLow,"F\t\tjustRight : ",justRight,"F\t\ttoHigh : ",tooHigh,"F")                       # if no monitor display
#                 print("tempC : ", temp, "C\t\ttempF : ",CtoF(temp),"F\t\tHumidity =", hum,"%\r\n")
                
#                 lastHum = hum          # save temp & humidity values so that there is no update to the RGB LCD
# #                 ftemp = CtoF(temp)     # unless the value changes
#                 lastTemp = temp       # this reduces the flashing of the display
#                 # print "ftemp = ",ftemp,"  temp = ",temp   # this was just for test and debug
                
#                 bgList = calcBG(temp)           # Calculate background colors
                
#                 t = str(temp)   # "stringify" the display values
#                 h = str(hum)
#                 # print "(",bgList[0],",",bgList[1],",",bgList[2],")"   # this was to test and debug color value list
#                 setRGB(bgList[0],bgList[1],bgList[2])   # parse our list into the color settings
#                 setText("Temp:" + t + "C      " + "Humidity :" + h + "%") # update the RGB LCD display
                
#     except (IOError,TypeError) as e:
#         print("Error" + str(e))
    

